In [28]:
import pandas as pd
from datetime import datetime
from data_tweak.converters import convert_race, convert_sex, convert_datetime

merged_chunks = []

cols = ["assignment_id","unit","watch","beat","car_number","shift_start","start_time","shift_end","end_time","off_last_name","off_first_name","off_middle_initial","off_rank","off_star","off_race","off_year_of_birth","off_appointed","is_working","absence_code","absence_description","off_sex","rdesc","date","year","shift_end_corrected","shift_time", "off_uniq_id"]

officers_prof_df = pd.read_csv("../files/profiles/officers_index_step_4.csv")
join_key = ["off_last_name","off_first_name","off_middle_initial","off_sex","off_year_of_birth","off_appointed"]
convert_sex(officers_prof_df, ["off_sex"])
convert_race(officers_prof_df, ["off_race"])

with pd.read_csv("../files/events/assignment_cleaned.csv", chunksize=10000) as reader:
  for chunk in reader:
    # convert_datetime(chunk, ["shift_start"], format="%Y-%m-%d %H:%M:%S")
    chunk.insert(0, "assignment_id", chunk.index)
    chunk.reset_index(drop=True, inplace=True)
    convert_sex(chunk, ["off_sex"])
    convert_race(chunk, ["off_race"])
    
    merged_chunk = pd.merge(chunk, officers_prof_df, on=join_key, suffixes=("", "_roster"), how="left")
    merged_chunk.sort_values(by=["shift_start", "assignment_id", "off_last_name", "off_first_name", "off_middle_initial"], inplace=True)
    merged_chunks.append(merged_chunk.loc[:, cols])

merged_df = pd.concat(merged_chunks)
grouped = merged_df.groupby("assignment_id").size().sort_values(ascending=False).to_frame("size").reset_index()
grouped[grouped["size"] > 1]


,assignment_id,size


In [29]:
merged_df.to_csv("../files/events/officer_id_merged/assignments/filter_1/assignments_officer_uid_merged_new.csv")

In [30]:
merged_df[merged_df["off_uniq_id"].isna()]

,assignment_id,unit,watch,beat,car_number,shift_start,start_time,shift_end,end_time,off_last_name,...,is_working,absence_code,absence_description,off_sex,rdesc,date,year,shift_end_corrected,shift_time,off_uniq_id
